In [ ]:
# %% [셀 1] 환경 설정 및 라이브러리 임포트
# 목적: 시각화 폰트, 경고 설정, 결과 저장 경로 준비
import os, platform, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import combinations

# 선택적: ML/설명가능성 패키지 (없으면 자동 스킵)
try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (11, 7)

# 결과물 저장 폴더
FIG_DIR = "figs"
REPORT_DIR = "reports"
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

print("환경 준비 완료")


In [ ]:
# %% [셀 2] 데이터 로드 및 기본 전처리
# 목적: 파일 로드, 핵심 수치형 정제, 컬럼 매핑 준비
FILE = "merged_data.csv"  # 필요시 교체

# 유연한 인코딩 로드
try:
    df = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False)
except UnicodeError:
    df = pd.read_csv(FILE, encoding="utf-8", low_memory=False)

print(f"데이터 파일 '{FILE}' 로드 완료. 원본 크기: {df.shape}")

# 핵심 수치형 정리
for col in ["사망자수", "중상자수", "경상자수"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=['사망자수', '중상자수', '경상자수']).copy()
df[['사망자수','중상자수','경상자수']] = df[['사망자수','중상자수','경상자수']].fillna(0)

# 유연한 컬럼명 매핑 테이블
col_map = {
    '사고월': ['사고월', '월'],
    '운전자연령': ['일당운전자연령', '가해운전자.연령대', '가해운전자연령', '운전자연령'],
    '사고장소': ['사고장소', '사고지역', '시군구', '시군구_only'],
    '사고유형': ['사고유형1', '사고유형'],
    '도로형태': ['도로형태'],
    '사고시각': ['사고시각'],
    '노면상태': ['노면상태'],
    '성별': ['일당운전자성별', '가해운전자.성별', '운전자성별'],
    '도로종류': ['도로종류'],
    '법규위반': ['법규위반'],
    '기상상태': ['기상상태'],
    '사고일자': ['사고일자_날짜', '발생년월_dt', '사고일자', '발생년월']
}

def get_col(df_, candidates):
    for name in candidates:
        if name in df_.columns:
            return name
    return None

print("기본 전처리 및 컬럼 매핑 준비 완료")


In [ ]:
# %% [셀 3] 안전 파서: '사고월' → '계절'
month_col = get_col(df, col_map['사고월'])

if month_col:
    df[month_col] = pd.to_numeric(df[month_col], errors='coerce').astype('Int64')

    def get_season(m):
        if pd.isna(m): return np.nan
        m = int(m)
        if m in (3,4,5):  return '봄'
        if m in (6,7,8):  return '여름'
        if m in (9,10,11): return '가을'
        return '겨울'

    df['계절'] = df[month_col].apply(get_season)
    print(f"'{month_col}' 기반 '계절' 생성 완료.")
else:
    print("계절 생성 불가: '사고월'/'월' 컬럼 없음.")


In [ ]:
# %% [셀 4] 안전 파서: '사고시각' → '사고시간대'
time_col = get_col(df, col_map['사고시각'])

def safe_hour(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # HHMM 순수숫자
    if s.isdigit() and len(s) in (3,4):
        try:
            h = int(s[:-2]) if len(s) == 4 else int(s[0])
            return h if 0 <= h <= 23 else np.nan
        except:
            return np.nan
    # HH, HH.0 등
    try:
        h = int(float(s))
        return h if 0 <= h <= 23 else np.nan
    except:
        return np.nan

if time_col:
    hrs = df[time_col].apply(safe_hour)

    def get_time_slot(h):
        if pd.isna(h): return np.nan
        h = int(h)
        if 0 <= h <= 5:   return '심야(0~5시)'
        if 6 <= h <= 9:   return '출근(6~9시)'
        if 10 <= h <= 17: return '낮(10~17시)'
        if 18 <= h <= 21: return '퇴근(18~21시)'
        return '밤(22~23시)'

    df['사고시간대'] = hrs.apply(get_time_slot)
    print(f"'{time_col}' 기반 '사고시간대' 생성 완료.")
else:
    print("시간대 생성 불가: '사고시각' 컬럼 없음.")


In [ ]:
# %% [셀 5] 안전 파서: '운전자연령' → '운전자연령대'
age_col = get_col(df, col_map['운전자연령'])

def parse_age(v):
    if pd.isna(v): return np.nan
    s = str(v).strip()
    if s.isdigit(): return int(s)
    s = s.replace(' ', '')
    for token in ['세', '살', '이상', '이하', '대']:
        s = s.replace(token, '')
    try:
        return int(float(s))
    except:
        return np.nan

def to_age_group(age):
    if pd.isna(age): return np.nan
    a = int(age)
    if a < 30: return '20대이하'
    if a < 40: return '30대'
    if a < 50: return '40대'
    if a < 60: return '50대'
    return '60대이상'

if age_col:
    ages = df[age_col].apply(parse_age)
    df['운전자연령대'] = ages.apply(to_age_group)
    print(f"'{age_col}' 기반 '운전자연령대' 생성 완료.")
else:
    print("연령대 생성 불가: 연령 컬럼 없음.")


In [ ]:
# %% [셀 6] 시공간 정교화: 날짜 파싱 및 파생
date_col = get_col(df, col_map['사고일자'])

def parse_date_safe(s):
    if pd.isna(s): return pd.NaT
    # 'YYYY-MM-DD', 'YYYYMM', 'YYYY.MM', 'YYYY/MM' 등 유연 처리
    v = str(s).strip()
    try:
        # 일반 날짜
        return pd.to_datetime(v, errors='coerce', format=None)
    except:
        pass
    # 연월만 있는 경우엔 1일로 보정
    for sep in ['.', '-', '/', ' ']:
        parts = v.split(sep)
        if len(parts) == 2 and all(p.isdigit() for p in parts):
            try:
                y, m = int(parts[0]), int(parts[1])
                return pd.to_datetime(f"{y:04d}-{m:02d}-01", errors='coerce')
            except:
                return pd.NaT
    # 순수 연월(YYYYMM)
    if v.isdigit() and len(v) == 6:
        try:
            y, m = int(v[:4]), int(v[4:])
            return pd.to_datetime(f"{y:04d}-{m:02d}-01", errors='coerce')
        except:
            return pd.NaT
    return pd.to_datetime(v, errors='coerce')

if date_col:
    dt = df[date_col].apply(parse_date_safe)
    df['사고일자_dt'] = dt
    df['요일'] = df['사고일자_dt'].dt.day_name()
    df['주말여부'] = df['요일'].isin(['Saturday', 'Sunday']).astype(int)
    df['분기'] = df['사고일자_dt'].dt.quarter
    # 러시아워 플래그
    df['러시아워'] = df['사고시간대'].isin(['출근(6~9시)', '퇴근(18~21시)']).astype(int)
    print(f"'{date_col}' 기반 시공간 파생 생성 완료.")
else:
    print("시공간 파생 일부 제한: 날짜 컬럼 없음.")


In [ ]:
# %% [셀 7] 통합지수 계산 및 도로별 집계 병합
df["다발도"] = (df["사망자수"].gt(0)).astype(float) * 1.0 \
             + (df["중상자수"].gt(0)).astype(float) * 0.7 \
             + (df["경상자수"].gt(0)).astype(float) * 0.3
df["심각도"] = df["사망자수"] * 1.0 + df["중상자수"] * 0.7 + df["경상자수"] * 0.3
df["통합지수"] = df["다발도"] * 0.4 + df["심각도"] * 0.6
print("통합지수 계산 완료. (정규화 없음)")

place_col = get_col(df, col_map['사고장소'])
if place_col:
    agg_place = (
        df.groupby(place_col, as_index=False)['통합지수']
          .sum()
          .rename(columns={'통합지수': '통합지수_합계'})
    )
    df_ = df.merge(agg_place, on=place_col, how='left')
    print(f"도로별 집계 병합 완료. df_ shape: {df_.shape}")
else:
    print("도로별 집계 불가: '사고장소'/'사고지역'/'시군구' 중 해당 없음.")
    df_ = pd.DataFrame()


In [ ]:
# %% [셀 8] 범주 정제 및 통합
violation_raw = get_col(df_, col_map['법규위반']) if not df_.empty else None

if violation_raw:
    violation_map = {
        '과속': '과속',
        '제한속도위반': '과속',
        '신호위반': '신호/통행위반',
        '통행금지위반': '신호/통행위반',
        '보행자보호의무위반': '신호/통행위반',
        '중앙선침범': '차로/진로위반',
        '진로변경위반': '차로/진로위반',
        '차로위반': '차로/진로위반',
        '안전거리미확보': '안전거리미확보'
    }
    df_['법규위반_대분류'] = df_[violation_raw].map(violation_map).fillna('기타')
    print("법규위반 대분류 생성 완료.")
else:
    print("법규위반 통합 생략: 해당 컬럼 없음.")


In [ ]:
# %% [셀 9] 통계 유틸 + 신뢰구간 + 요약 텍스트
def _fmt(x, fmt=".3f"):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)): return "nan"
        return format(x, fmt)
    except Exception:
        return str(x)

def mean_ci(series, alpha=0.05):
    arr = np.array(series.dropna())
    n = len(arr)
    if n == 0:
        return np.nan, (np.nan, np.nan), n
    m = np.mean(arr)
    se = stats.sem(arr)
    if n > 1 and np.isfinite(se):
        ci_low, ci_high = stats.t.interval(1 - alpha, n - 1, loc=m, scale=se)
    else:
        ci_low, ci_high = np.nan, np.nan
    return m, (ci_low, ci_high), n

def kruskal_overall(tmp: pd.DataFrame, col: str, metric: str):
    if (col not in tmp.columns) or (metric not in tmp.columns):
        return np.nan, np.nan, 0
    groups = [g[metric].dropna().values for _, g in tmp.groupby(col)]
    groups = [g for g in groups if len(g) > 0]
    if len(groups) < 2:
        return np.nan, np.nan, len(groups)
    H, p = stats.kruskal(*groups)
    return H, p, len(groups)

def generate_summary_with_ci(df_source: pd.DataFrame, col: str, metric: str, topn=3):
    if col not in df_source.columns or metric not in df_source.columns:
        return f"[{col}] 분석 불가: 필요한 컬럼 없음."
    tmp = df_source.dropna(subset=[col])[[col, metric]].copy()
    if tmp.empty:
        return f"[{col}] 분석 불가: 유효 데이터 없음."

    # 그룹별 평균 및 95% CI
    rows = []
    for k, g in tmp.groupby(col):
        mean_val, (ci_l, ci_h), n = mean_ci(g[metric])
        rows.append((k, mean_val, ci_l, ci_h, n))
    stats_df = pd.DataFrame(rows, columns=['그룹','평균','CI하한','CI상한','표본수']).sort_values('평균', ascending=False)

    # 전체 유의성
    H, p, _ = kruskal_overall(tmp.rename(columns={col: "grp", metric: "val"}), "grp", "val")

    # 요약 문구
    head = stats_df.head(min(topn, len(stats_df)))
    tail = stats_df.tail(min(topn, len(stats_df)))
    def to_items(df_):
        return ", ".join([f"{r['그룹']}({_fmt(r['평균'])}, [{_fmt(r['CI하한'])}, {_fmt(r['CI상한'])}], n={int(r['표본수'])})" for _, r in df_.iterrows()])
    text = []
    text.append(f"[{col}] Kruskal–Wallis H={_fmt(H)}, p={_fmt(p, '.3g')}")
    text.append(f"- 상위 {len(head)}: {to_items(head)}")
    text.append(f"- 하위 {len(tail)}: {to_items(tail)}")
    return "\n".join(text), stats_df


## 시각화

In [ ]:
# %% [셀 10] 단일 변수 시각화 + 신뢰구간 표 + 유의미성 문구
report_lines = []

def save_group_ci_table(stats_df, name):
    path = os.path.join(REPORT_DIR, f"ci_{name}.csv")
    stats_df.to_csv(path, index=False, encoding="utf-8-sig")
    return path

if not df_.empty:
    print("\n--- 단일 변수 시각화 및 통계 분석 ---")

    # 1) 사고장소별 통합지수 합계 Top 10
    place_col = get_col(df_, col_map['사고장소'])
    if place_col:
        top10 = df_.groupby(place_col)['통합지수'].sum().nlargest(10).reset_index()
        plt.figure(figsize=(12, 8))
        sns.barplot(x='통합지수', y=place_col, data=top10)
        plt.title('사고장소별 통합지수 합계 Top 10')
        plt.xlabel('통합지수 합계'); plt.ylabel('사고장소')
        plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'place_top10_totalrisk.png')); plt.show()

        text, _ = generate_summary_with_ci(df_, place_col, '통합지수', topn=3)
        print(text); report_lines += ["[그림] 사고장소 Top10", text, ""]
    else:
        print("[경고] 사고장소 컬럼 없음.")

    # 2) 도로종류별 통합지수 분포(Box) + CI 표
    road_kind_col = get_col(df_, col_map['도로종류'])
    if road_kind_col:
        plt.figure(figsize=(12, 6))
        sns.boxplot(x=df_[road_kind_col], y=df_['통합지수'])
        plt.title('도로종류별 통합지수 분포')
        plt.xlabel('도로종류'); plt.ylabel('통합지수')
        plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'box_roadkind_totalrisk.png')); plt.show()

        text, stats_df = generate_summary_with_ci(df_, road_kind_col, '통합지수', topn=3)
        path = save_group_ci_table(stats_df, "roadkind")
        print(text); report_lines += ["[그림] 도로종류 분포", text, f"CI 표: {path}", ""]
    else:
        print("[경고] 도로종류 컬럼 없음.")

    # 3) 월별 사고 건수 추이(Line)
    month_col = get_col(df_, col_map['사고월'])
    if month_col:
        monthly_cnt = df_.groupby(month_col).size()
        plt.figure(figsize=(10, 6))
        sns.lineplot(x=monthly_cnt.index, y=monthly_cnt.values, marker='o')
        plt.title('월별 사고 건수 추이')
        plt.xlabel('월'); plt.ylabel('사고 건수')
        plt.xticks(monthly_cnt.index); plt.grid(True)
        plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'line_monthly_count.png')); plt.show()
        report_lines += ["[그림] 월별 사고 건수", "설명: 추세 확인(평균 비교 검정 대상 아님).", ""]
    else:
        print("[경고] 사고월 컬럼 없음.")

    # 4) 사고시간대별 평균 통합지수 + CI
    if '사고시간대' in df_.columns:
        text, stats_df = generate_summary_with_ci(df_, '사고시간대', '통합지수', topn=3)
        path = save_group_ci_table(stats_df, "timeslot")
        plt.figure(figsize=(10, 6))
        order = stats_df.sort_values('평균', ascending=False)['그룹']
        sns.barplot(x='그룹', y='평균', data=stats_df, order=order)
        plt.title('사고시간대별 평균 통합지수 (점추정)')
        plt.xlabel('사고시간대'); plt.ylabel('평균 통합지수')
        plt.xticks(rotation=0)
        plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'bar_timeslot_meanrisk.png')); plt.show()
        print(text); report_lines += ["[그림] 사고시간대 평균", text, f"CI 표: {path}", ""]
    else:
        print("[경고] 사고시간대 컬럼 없음.")

    # 5) 법규위반 대분류별 평균 통합지수 + CI
    if '법규위반_대분류' in df_.columns:
        text, stats_df = generate_summary_with_ci(df_, '법규위반_대분류', '통합지수', topn=3)
        path = save_group_ci_table(stats_df, "violation_grouped")
        plt.figure(figsize=(14, 7))
        order = stats_df.sort_values('평균', ascending=False)['그룹']
        sns.barplot(x='그룹', y='평균', data=stats_df, order=order)
        plt.title('법규위반 대분류별 평균 통합지수 (점추정)')
        plt.xlabel('법규위반 대분류'); plt.ylabel('평균 통합지수')
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'bar_violation_grouped_meanrisk.png')); plt.show()
        print(text); report_lines += ["[그림] 법규위반 대분류 평균", text, f"CI 표: {path}", ""]
    else:
        print("[경고] 법규위반 대분류 컬럼 없음.")
else:
    print("df_ 비어있음: 단일 변수 시각화 생략")


In [ ]:
# %% [셀 11] 복합 변수 히트맵 + 행/열 유의성 요약
def heatmap_mean(df_in, idx, col, val='통합지수', title='', fname='heatmap.png', cmap='Oranges'):
    pv = df_in.pivot_table(index=idx, columns=col, values=val, aggfunc='mean')
    if pv.empty: return False
    plt.figure(figsize=(15, 8))
    sns.heatmap(pv, cmap=cmap, annot=True, fmt=".2f", linewidths=.5, linecolor='white')
    plt.title(title); plt.xlabel(col); plt.ylabel(idx)
    plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, fname)); plt.show()
    return True

def rowcol_significance(df_in, row, col, metric='통합지수'):
    pv = df_in.pivot_table(index=row, columns=col, values=metric, aggfunc='mean')
    rows, cols = [], []
    if not pv.empty:
        for r in pv.index:
            sub = df_in.loc[df_in[row] == r, [col, metric]].rename(columns={col: "grp", metric: "val"})
            H, p, _ = kruskal_overall(sub, "grp", "val")
            rows.append((r, p))
        for c in pv.columns:
            sub = df_in.loc[df_in[col] == c, [row, metric]].rename(columns={row: "grp", metric: "val"})
            H, p, _ = kruskal_overall(sub, "grp", "val")
            cols.append((c, p))
    row_sig = sum(1 for _, p in rows if not np.isnan(p) and p < 0.05)
    col_sig = sum(1 for _, p in cols if not np.isnan(p) and p < 0.05)
    return row_sig, len(rows), col_sig, len(cols)

if not df_.empty:
    # 1) 운전자연령대 × 법규위반 대분류
    if ('운전자연령대' in df_.columns) and ('법규위반_대분류' in df_.columns):
        if heatmap_mean(df_, '운전자연령대', '법규위반_대분류', '통합지수',
                        '운전자연령대 × 법규위반 대분류 평균 통합지수', 'hm_age_violation.png', 'Oranges'):
            rs, rn, cs, cn = rowcol_significance(df_, '운전자연령대', '법규위반_대분류')
            line = f"[운전자연령대×법규위반_대분류] 행 유의 {rs}/{rn}, 열 유의 {cs}/{cn}"
            print(line); report_lines += ["[그림] 운전자연령대×법규위반", line, ""]

    # 2) 계절 × 기상상태
    if ('계절' in df_.columns) and (get_col(df_, col_map['기상상태']) is not None):
        wcol = get_col(df_, col_map['기상상태'])
        if heatmap_mean(df_, '계절', wcol, '통합지수',
                        '계절 × 기상상태 평균 통합지수', 'hm_season_weather.png', 'Blues'):
            rs, rn, cs, cn = rowcol_significance(df_, '계절', wcol)
            line = f"[계절×{wcol}] 행 유의 {rs}/{rn}, 열 유의 {cs}/{cn}"
            print(line); report_lines += ["[그림] 계절×기상상태", line, ""]

    # 3) 사고유형 × 법규위반 대분류
    type_col = get_col(df_, col_map['사고유형'])
    if (type_col is not None) and ('법규위반_대분류' in df_.columns):
        if heatmap_mean(df_, type_col, '법규위반_대분류', '통합지수',
                        '사고유형 × 법규위반 대분류 평균 통합지수', 'hm_type_violation.png', 'Greens'):
            rs, rn, cs, cn = rowcol_significance(df_, type_col, '법규위반_대분류')
            line = f"[{type_col}×법규위반_대분류] 행 유의 {rs}/{rn}, 열 유의 {cs}/{cn}"
            print(line); report_lines += ["[그림] 사고유형×법규위반", line, ""]

    # 4) 사고유형 × 도로형태
    road_type_col = get_col(df_, col_map['도로형태'])
    if (type_col is not None) and (road_type_col is not None):
        if heatmap_mean(df_, type_col, road_type_col, '통합지수',
                        '사고유형 × 도로형태 평균 통합지수', 'hm_type_road.png', 'Purples'):
            rs, rn, cs, cn = rowcol_significance(df_, type_col, road_type_col)
            line = f"[{type_col}×{road_type_col}] 행 유의 {rs}/{rn}, 열 유의 {cs}/{cn}"
            print(line); report_lines += ["[그림] 사고유형×도로형태", line, ""]

    # 5) 사고시간대 × 도로형태
    if ('사고시간대' in df_.columns) and (road_type_col is not None):
        if heatmap_mean(df_, '사고시간대', road_type_col, '통합지수',
                        '사고시간대 × 도로형태 평균 통합지수', 'hm_time_road.png', 'PuRd'):
            rs, rn, cs, cn = rowcol_significance(df_, '사고시간대', road_type_col)
            line = f"[사고시간대×{road_type_col}] 행 유의 {rs}/{rn}, 열 유의 {cs}/{cn}"
            print(line); report_lines += ["[그림] 사고시간대×도로형태", line, ""]
else:
    print("df_ 비어있음: 복합 히트맵 생략")


In [ ]:
# %% [셀 12] 특징 조합 유의성 분석
# 질문 1: 상위 하위 100개의 특징들 중 몇 개의 조합 이상일 때 유의미하게 증감이 나타나는지
# 질문 2: 어떤 조합이 유의미한 증감을 보이는지

# 분석에 사용할 특징 후보 (실제 존재하는 것만 사용)
candidate_feats = []
for k in ['계절', '운전자연령대', '사고시간대', '도로종류', '법규위반_대분류', '기상상태', '도로형태', '노면상태', '성별', '주말여부', '분기', '러시아워']:
    if k in df_.columns:
        candidate_feats.append(k)

def combo_significance(df_in, feature_list, metric='통합지수', max_combos_per_r=1000):
    results = []
    for r in range(1, len(feature_list)+1):
        combos = list(combinations(feature_list, r))
        # 너무 많은 조합이면 상위 일부만 샘플 (범주 수가 큰 피처 우선)
        if len(combos) > max_combos_per_r:
            # 간단한 휴리스틱: 이름 길이 기준 정렬 후 앞부분 샘플
            combos = sorted(combos, key=lambda x: sum(len(c) for c in x))[:max_combos_per_r]
        for combo in combos:
            grp = df_in.groupby(list(combo))[metric]
            if grp.ngroups < 2:
                continue
            groups = [g.dropna().values for _, g in grp]
            if all(len(g) > 0 for g in groups):
                try:
                    H, p = stats.kruskal(*groups)
                except Exception:
                    H, p = np.nan, np.nan
            else:
                H, p = np.nan, np.nan
            results.append((combo, r, grp.ngroups, H, p))
    return pd.DataFrame(results, columns=['특징조합','조합크기','그룹수','H','p'])

if not df_.empty and len(candidate_feats) >= 1:
    res_df = combo_significance(df_, candidate_feats, '통합지수', max_combos_per_r=2000)
    # 조합 크기별 유의 비율
    size_sig_ratio = res_df.groupby('조합크기').apply(lambda g: (g['p'] < 0.05).mean()).rename('p<0.05 비율')
    print("[조합 크기별 유의 비율]")
    print(size_sig_ratio)

    # 유의미 조합 Top 30
    sig_combos = res_df[res_df['p'] < 0.05].sort_values('p').head(30).copy()

    # 조합별 효과 크기 대략 보기: 그룹별 평균 범위(최대-최소)
    effect_rows = []
    for combo in sig_combos['특징조합']:
        means = df_.groupby(list(combo))['통합지수'].mean()
        rng = means.max() - means.min()
        effect_rows.append((combo, rng, len(means)))
    effect_df = pd.DataFrame(effect_rows, columns=['특징조합','평균범위(대략효과)','그룹수']).sort_values('평균범위(대략효과)', ascending=False)

    # 저장
    path_all = os.path.join(REPORT_DIR, "combo_significance_all.csv")
    path_sig = os.path.join(REPORT_DIR, "combo_significance_sigTop30.csv")
    path_eff = os.path.join(REPORT_DIR, "combo_effectsize_sigTop30.csv")
    res_df.to_csv(path_all, index=False, encoding="utf-8-sig")
    sig_combos.to_csv(path_sig, index=False, encoding="utf-8-sig")
    effect_df.to_csv(path_eff, index=False, encoding="utf-8-sig")

    report_lines += [
        "[특징 조합 유의성]",
        f"- 조합 크기별 p<0.05 비율:\n{size_sig_ratio.to_string()}",
        f"- 유의미 조합 Top30: {path_sig}",
        f"- 유의미 조합 효과크기 Top30(평균범위): {path_eff}",
        ""
    ]
else:
    print("특징 조합 분석 생략: df_ 비어있거나 특징 후보 부족")


In [ ]:
# %% [셀 13] ML 고도화와 해석: XGBoost + SHAP (없으면 RF)
if not df_.empty:
    print("\n--- 머신러닝 분석 ---")
    target_col = '통합지수'
    raw_features = ['계절', '운전자연령대', '사고시간대', '도로종류', '법규위반_대분류',
                    '기상상태', '도로형태', '노면상태', '성별', '주말여부', '분기', '러시아워']

    feature_cols = [c for c in raw_features if c in df_.columns]
    if len(feature_cols) == 0:
        print("머신러닝 생략: 사용 가능 특징 없음")
    else:
        use_cols = feature_cols + [target_col]
        df_ml = df_[use_cols].dropna().copy()
        if len(df_ml) > 0:
            cat_cols = [c for c in feature_cols if df_ml[c].dtype == 'object']
            df_ml_enc = pd.get_dummies(df_ml, columns=cat_cols, drop_first=True)
            X = df_ml_enc.drop(columns=[target_col])
            y = df_ml_enc[target_col]

            MAX_ML_ROWS = 300_000
            if len(X) > MAX_ML_ROWS:
                X = X.sample(MAX_ML_ROWS, random_state=42)
                y = y.loc[X.index]

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model_name = ""
            if HAS_XGB:
                model = xgb.XGBRegressor(n_estimators=400, max_depth=6, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, n_jobs=-1, random_state=42)
                model_name = "XGBoost"
            else:
                model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
                model_name = "RandomForest"

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            r2 = r2_score(y_test, y_pred)
            print(f"{model_name} 성능: RMSE={rmse:.4f}, R2={r2:.4f}")

            # 변수 중요도 Top 20
            try:
                importances = model.feature_importances_
                fi = pd.Series(importances, index=X.columns).sort_values(ascending=False)
                plt.figure(figsize=(12, 8))
                sns.barplot(x=fi.values[:20], y=fi.index[:20])
                plt.title(f'{model_name} 변수 중요도 Top 20'); plt.xlabel('중요도'); plt.ylabel('변수')
                plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, f'feature_importance_top20_{model_name}.png')); plt.show()
            except Exception:
                pass

            # SHAP 해석 (가능할 때만)
            if HAS_SHAP and HAS_XGB and model_name == "XGBoost":
                explainer = shap.TreeExplainer(model)
                # 큰 데이터 방지: 일부 샘플
                sample_idx = np.random.choice(X.shape[0], size=min(20000, X.shape[0]), replace=False)
                X_sample = X.iloc[sample_idx]
                shap_values = explainer.shap_values(X_sample)
                try:
                    shap.summary_plot(shap_values, X_sample, max_display=20, show=False)
                    plt.tight_layout(); plt.savefig(os.path.join(FIG_DIR, 'shap_summary.png')); plt.close()
                except Exception:
                    pass

            report_lines += [
                "[ML] 모델 결과",
                f"- 모델: {model_name}",
                f"- 성능: RMSE={rmse:.4f}, R2={r2:.4f}",
                ""
            ]
        else:
            print("머신러닝 생략: 유효 행 없음")
else:
    print("df_ 비어있음: 머신러닝 생략")


In [ ]:
# %% [셀 14] 리포트 저장 및 해석 가이드
report_path = os.path.join(REPORT_DIR, "analysis_summary.md")
with open(report_path, "w", encoding="utf-8") as f:
    f.write("# 분석 요약 및 유의미성 검증 결과\n\n")
    f.write("본 리포트는 비모수 검정(Kruskal–Wallis)과 그룹별 95% 신뢰구간을 포함합니다.\n")
    f.write("특징 조합의 유의성 탐색을 통해 '최소 몇 개의 특징을 묶어야\n")
    f.write("유의미한 증감이 나타나는지'와 '어떤 조합이 유의미한지'를 제시합니다.\n\n")
    f.write("\n".join(report_lines))

print(f"리포트 저장 완료: {report_path}")

print("""
[의미성 검증 해석 가이드]
- Kruskal–Wallis p<0.05: 범주 간 차이가 통계적으로 유의할 가능성이 높음(비모수).
- 그룹별 95% 신뢰구간: 평균의 불확실성 범위 제시 → 구간 겹침 여부로 직관적 비교 가능.
- 특징 조합 분석: 조합 크기 증가에 따라 p<0.05 비율이 상승하면,
  해당 데이터에서는 '그 정도 차원수' 이상에서 상호작용 패턴이 드러난다는 의미.
- 효과크기(평균범위): 유의미 조합 중 평균의 최대–최소 범위가 큰 조합은 정책 타깃팅 우선 후보.

[발전 방향]
- 노출 보정(통행량): 현재 가용치 없음 → 확보 시 통합지수 합계 대비 노출로 '위험률' 산출 권장.
- 혼합효과모형: 요일/월/계절 등 시계열 교란을 임의효과로 모델링해 순효과 분리.
- 범주군 축소/병합: 희소 범주를 대분류로 통합해 신뢰구간 폭 축소.
- 정책 매핑: 상위 위험 장소 Top N × 주요 법규위반 대분류 × 도로형태 조합으로 맞춤 개입안 설계.
""")
